# Process training Data to train a RNN

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pprint as pprint
import math
import time
import array 

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [3]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import os
import json
import pickle
import sklearn
from sklearn import neighbors

### 1. Pre-process data

In [4]:

def get_data (file):
    raw_dataset = pd.read_csv(file)
    dataset = raw_dataset.copy()
    training_size = len(dataset)
    N_CATEGORIES = 28

    dataset["A0"] = np.nan
    dataset["A1"] = np.nan

    for index, row in dataset.iterrows():
        CS_array =np.array(eval(row["current_state"]))
        dataset["current_state"][index] = CS_array
        GS_array =np.array(eval(row["goal_state"]))
        dataset["goal_state"][index] = GS_array
        GS_array =np.array(eval(row["initial_state"]))
        dataset["initial_state"][index] = GS_array
        GS_array =np.array(eval(row["out_action"]))
        dataset["out_action"][index] = GS_array

    # Usar estado final 
    dataset["Difference"] =   dataset["goal_state"]-dataset["current_state"]

    dataset[["C0",'C1',"C2","C3",'C4',"C5"]] = pd.DataFrame(dataset.current_state.values.tolist(), index= dataset.index)
    dataset[["D0",'D1',"D2","D3",'D4',"D5"]] = pd.DataFrame(dataset.Difference.values.tolist(), index= dataset.index)
    dataset[["A0","A1"]] = pd.DataFrame(dataset.out_action.values.tolist(), index= dataset.index)

    return dataset

In [5]:
def add_end_state(dataset_short):
    extended_dataset = dataset_short.copy()
    exit_state = pd.DataFrame([{"id_camino":0,"D0":0,'D1':0,"D2":0,"D3":0,'D4':0,"D5":0,"A0":-0,"A1":-0}])
    for camino in extended_dataset["id_camino"].unique():
        exit_state["id_camino"] = camino
        extended_dataset = extended_dataset.append(exit_state,ignore_index=True)

    return extended_dataset

In [6]:
def extend_dataset(dataset_short, N_STEPS):
    extended_dataset = dataset_short.copy()
    last = extended_dataset.iloc[-1,0]
    i=1
    for camino in extended_dataset["id_camino"].unique():
        len_camino = len(extended_dataset[extended_dataset["id_camino"]==camino])
        if(len_camino>N_STEPS):
            tail= extended_dataset[extended_dataset["id_camino"]==camino][len_camino-N_STEPS:]
            tail["id_camino"]= last+i
            i=i+1
            extended_dataset = extended_dataset.append(tail,ignore_index=True)

    return extended_dataset

## Action treatment

In [7]:
def closest_category(X, pred):
    tree = sklearn.neighbors.KDTree(X, leaf_size=2)
    dist, ind = tree.query(pred, k=1)
    return ind

In [8]:
def class_to_action (class_int):
    with open('./data/class_to_action (28).json') as json_file:
        CtA = json.load(json_file)
        
    action = CtA[class_int]
    return action

# Normalize the data

In [9]:
def norm(x, train_stats):
  #return (x - train_stats['mean']) / train_stats['std']
  return (x) / train_stats['std']




In [10]:
def denorm(x, train_stats):
  #return (x - train_stats['mean']) / train_stats['std']
  return (x) * train_stats['std']


In [11]:
def normalize_data(dataset_short,action_vector):
    reduced_dataset = dataset_short.copy()
    reduced_dataset.pop("A0")
    reduced_dataset.pop("A1")
    reduced_dataset.pop("id_camino")
    data_stats = reduced_dataset.describe()
    data_stats = data_stats.transpose()

    action_vector.append("id_camino")
    normed_data = norm(reduced_dataset, data_stats)
    normed_dataset = normed_data.join(dataset_short[action_vector])

    return normed_dataset, data_stats

## Pad the data

In [12]:
def padding(normed_dataset,N_STEPS, position):
    PAD_LEN = N_STEPS+1
    padded_data = []
    for camino in normed_dataset["id_camino"].unique():
        group = normed_dataset[normed_dataset["id_camino"]==camino]
        group.pop("id_camino")
        padded_data.append(group.values)
    
    dataset2 = tf.keras.preprocessing.sequence.pad_sequences(padded_data, maxlen=PAD_LEN, dtype='float64', padding=position, truncating=position, value=0.0)

    return dataset2

## Group the data

In [13]:
def group_by_path(normed_dataset):
    dataset2 = []
    for camino in normed_dataset["id_camino"].unique():
        group = normed_dataset[normed_dataset["id_camino"]==camino]
        group.pop("id_camino")
        dataset2.append(group.values)
    
    return np.array(dataset2)

In [14]:
def multiplex_trajectories(dataset2):
    multitrajectory = dataset2.tolist()

    for trajectory in dataset2:
        len_trajectory = len(trajectory)
        for i in range(2,len_trajectory):
            subtrajectory= trajectory[0:i]
            # new_trajectory = np.expand_dims(subtrajectory, axis=0)
            # array_tuple = (multitrajectory, new_trajectory)
            # multitrajectory = np.vstack(  array_tuple )
            multitrajectory.append(subtrajectory)

    return np.asarray(multitrajectory)

In [15]:
file = "OSPA_training_data_short.csv"

In [16]:
STEPS = 50

dataset = get_data("./data/"+file)

dataset_distances= dataset[["id_camino","D0",'D1',"D2","D3",'D4',"D5","A0","A1"]]

exteneded_dataset= add_end_state(dataset_distances)

action_vector = []
normed_dataset, data_stats = normalize_data(exteneded_dataset, action_vector)

dataset2 = group_by_path(normed_dataset)

C:\Users\pasky\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\pasky\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\pasky\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\pasky\Anaconda3\lib\site-packages\ip

# Evaluate the data

In [17]:
from ast import literal_eval
def print_model_df_analysis(csv_name ="./data/model_data_csv.csv", percent_value=25/100.0):
    df = pd.read_csv(csv_name, converters={"target":literal_eval})
    df["abs_x"] = df["target"].apply(lambda x: abs(x[0]))
    df["abs_z"] = df["target"].apply(lambda x: abs(x[1]))

    return df.loc[df["abs_x"]*percent_value>df["abs_z"]]


In [18]:
def find_zvalue(x, sample_path, from_index, x_index=0, z_index=1):
    while x>sample_path[from_index][x_index]:
        from_index += 1
    prev_z = sample_path[from_index - 1][z_index]
    next_z = sample_path[from_index][z_index]

    prev_x = sample_path[from_index -1][x_index]
    next_x = sample_path[from_index][x_index]

    z_value = (next_z - prev_z) / (next_x-prev_x)*(x-prev_x)+prev_z
    return z_value, from_index

def get_mean_error(ospa_path, model_path, num=10, eps = 5):
    s0= ospa_path[0]
    sf = ospa_path[-1]
    x_values = np.linspace(s0[0]+eps, sf[0]-eps, num = num+1)

    min_ospa_index, min_model_index = 0, 0
    error=0
    for x in x_values:
        ospa_z, min_ospa_index = find_zvalue(x,ospa_path, min_ospa_index)
        try:
            model_z, min_model_index = find_zvalue(x, model_path, min_model_index)
            error += abs(model_z-ospa_z)
        except Exception as e:
            print(e)
    return error/(num+1)